In [1]:
import os
import pandas as pd
from datetime import datetime

# import tensorflow as tf
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator


def extract_scalars_from_event_file(event_file):
    event_acc = EventAccumulator(event_file)
    event_acc.Reload()
    

    tags = event_acc.Tags()["scalars"]

    all_scalars = {}
    for tag in tags:
        events = event_acc.Scalars(tag)
        values = [event.value for event in events]
        steps = [event.step for event in events]

        timestamps = [pd.to_datetime(event.wall_time, unit='s') for event in events]
        all_scalars[tag] = pd.DataFrame({"step": steps, "value": values, "timestamp": timestamps})

    return all_scalars


files_t = []
def event_files_to_dataframe(directory):
    all_scalars_dfs = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.startswith("events.out.tfevents"):
                event_file = os.path.join(root, file)

                file_from = pd.to_datetime(file.split(".")[3], unit='s')
                files_t.append(file_from)

                # print(file_from)
                if file_from < date_obj:
                    continue

                scalars = extract_scalars_from_event_file(event_file)
                for tag, df in scalars.items():
                    df["tag"] = tag
                    all_scalars_dfs.append(df)

    all_scalars_df = pd.concat(all_scalars_dfs, ignore_index=True)
    return all_scalars_df

df_all = []
for i in range(15):
    # Directory containing your event files
    log_dir = f"/home/<User>/repos/csg_is/fedstellar/app/logs/fedstellar_DFL_03_07_2024_14_38_33/metrics/participant_{i}"
    date_str = "02/07/2024 10:22:20" # minus two hours for timezone

    date_obj = datetime.strptime(date_str, "%d/%m/%Y %H:%M:%S")

    df = event_files_to_dataframe(log_dir)
    print(df.head())

    df.sort_values(by="timestamp", inplace=True)

    files_t.sort(reverse=True)
    print(files_t[0], files_t[-1])

    df[df["tag"]=='Test/Loss']["timestamp"].max()

    test_df = df[df["tag"].str.startswith("Test/")].groupby("tag").last().copy()
    print(test_df)
    test_df["participant"] = f"participant_{i}"
    df_all.append(test_df)

   step     value                     timestamp             tag
0  1980  9.000000 2024-07-03 12:45:58.702709504           Round
1   990  0.130849 2024-07-03 12:46:01.200684288       Test/Loss
2   990  0.898307 2024-07-03 12:46:01.200750080   Test/Accuracy
3   990  0.895253 2024-07-03 12:46:01.200773120  Test/Precision
4   990  0.898307 2024-07-03 12:46:01.200794112     Test/Recall
2024-07-03 12:46:31 2024-07-03 12:39:36
                step     value                     timestamp
tag                                                         
Test/Accuracy   1100  0.905027 2024-07-03 12:46:24.083625984
Test/F1Score    1100  0.895127 2024-07-03 12:46:24.083698432
Test/Loss       1100  0.184250 2024-07-03 12:46:24.083566080
Test/Precision  1100  0.910700 2024-07-03 12:46:24.083655424
Test/Recall     1100  0.905027 2024-07-03 12:46:24.083677952
   step     value                     timestamp         tag
0   458  0.260364 2024-07-03 12:42:03.950646528  Train/Loss
1   478  0.266583 2024-07-03 

In [2]:
concat_df = pd.concat(df_all)
concat_df

,step,value,timestamp,participant
tag,,,,
Test/Accuracy,1100,0.905027,2024-07-03 12:46:24.083625984,participant_0
Test/F1Score,1100,0.895127,2024-07-03 12:46:24.083698432,participant_0
Test/Loss,1100,0.184250,2024-07-03 12:46:24.083566080,participant_0
Test/Precision,1100,0.910700,2024-07-03 12:46:24.083655424,participant_0
Test/Recall,1100,0.905027,2024-07-03 12:46:24.083677952,participant_0
...,...,...,...,...
Test/Accuracy,1257,0.932995,2024-07-03 12:46:25.062329088,participant_14
Test/F1Score,1257,0.930887,2024-07-03 12:46:25.062420992,participant_14
Test/Loss,1257,0.116910,2024-07-03 12:46:25.062257408,participant_14


In [3]:
concat_df.groupby("tag").mean().round(4)

/tmp/ipykernel_361203/4276830878.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  concat_df.groupby("tag").mean().round(4)


,step,value
tag,,
Test/Accuracy,1231.5333,0.9058
Test/F1Score,1231.5333,0.8979
Test/Loss,1231.5333,0.1694
Test/Precision,1231.5333,0.9080
Test/Recall,1231.5333,0.9058


In [4]:
concat_df.to_csv("results_csvs/dfl_15_ring.csv")